# Imports

In [1]:
import pandas as pd
import numpy as np
import inflection
from ydata_profiling import ProfileReport

# Functions

In [2]:
def rename_columns(dataframe):
    df = dataframe.copy()
    title = lambda x: inflection.titleize(x)
    snakecase = lambda x: inflection.underscore(x)
    spaces = lambda x: x.replace(" ", "")
    cols_old = list(df.columns)
    cols_old = list(map(title, cols_old))
    cols_old = list(map(spaces, cols_old))
    cols_new = list(map(snakecase, cols_old))
    df.columns = cols_new
    return df

COUNTRIES = {
1: "India",
14: "Australia",
30: "Brazil",
37: "Canada",
94: "Indonesia",
148: "New Zeland",
162: "Philippines",
166: "Qatar",
184: "Singapure",
189: "South Africa",
191: "Sri Lanka",
208: "Turkey",
214: "United Arab Emirates",
215: "England",
216: "United States of America"}

def country_name(country_id):
    return COUNTRIES[country_id]


def create_price_tye(price_range):
    if price_range == 1:
        return "cheap"
    elif price_range == 2:
        return "normal"
    elif price_range == 3:
        return "expensive"
    else:
        return "gourmet"


COLORS = {
"3F7E00": "darkgreen",
"5BA829": "green",
"9ACD32": "lightgreen",
"CDD614": "orange",
"FFBA00": "red",
"CBCBC8": "darkred",
"FF7800": "darkred"}

def color_name(color_code):
    return COLORS[color_code]

# Data

In [3]:
df_raw=pd.read_csv('../data/zomato.csv')
df_raw.head(1)

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,Botswana Pula(P),1,0,0,0,3,4.6,3F7E00,Excellent,619


In [4]:
print(df_raw.shape,'\n')
print(df_raw.isna().sum(),'\n')
print(df_raw.dtypes)

(7527, 21) 

Restaurant ID            0
Restaurant Name          0
Country Code             0
City                     0
Address                  0
Locality                 0
Locality Verbose         0
Longitude                0
Latitude                 0
Cuisines                15
Average Cost for two     0
Currency                 0
Has Table booking        0
Has Online delivery      0
Is delivering now        0
Switch to order menu     0
Price range              0
Aggregate rating         0
Rating color             0
Rating text              0
Votes                    0
dtype: int64 

Restaurant ID             int64
Restaurant Name          object
Country Code              int64
City                     object
Address                  object
Locality                 object
Locality Verbose         object
Longitude               float64
Latitude                float64
Cuisines                 object
Average Cost for two      int64
Currency                 object
Has Table booking    

In [5]:
# profile = ProfileReport(df_raw, title="Profiling Report")
# profile.to_file('analysis.html')

In [6]:
# fixing columns names

df_raw=rename_columns(df_raw)
df_raw.columns

Index(['restaurant_id', 'restaurant_name', 'country_code', 'city', 'address',
       'locality', 'locality_verbose', 'longitude', 'latitude', 'cuisines',
       'average_cost_for_two', 'currency', 'has_table_booking',
       'has_online_delivery', 'is_delivering_now', 'switch_to_order_menu',
       'price_range', 'aggregate_rating', 'rating_color', 'rating_text',
       'votes'],
      dtype='object')

In [7]:
# drop all 15 nas

df_raw=df_raw.dropna(ignore_index=True)

In [8]:
# drop column with only one variable

df_raw=df_raw.drop(columns=['switch_to_order_menu'])

In [9]:
# replacing country codes with their names

list=[]
for index, row in df_raw.iterrows():
    codes=country_name(row['country_code'])
    list.append(codes)

df_raw['country']=list
len(df_raw['country'].unique())

15

In [10]:
# replacing price range values with strings 

df_raw['price_range']=df_raw['price_range'].apply(create_price_tye)
df_raw['price_range'].unique()

array(['expensive', 'gourmet', 'normal', 'cheap'], dtype=object)

In [11]:
# replacing hex color codes with the color names

list=[]
for index, row in df_raw.iterrows():
    codes=color_name(row['rating_color'])
    list.append(codes)

df_raw['rating_color']=list
df_raw['rating_color'].unique()

array(['darkgreen', 'green', 'lightgreen', 'orange', 'red', 'darkred'],
      dtype=object)

In [12]:
# creating another column to count different cuisines categories

list=df_raw['cuisines'].str.split(',').apply(len)
df_raw['cuisines_n°']=list
df_raw[['cuisines','cuisines_n°']].head(5)

,cuisines,cuisines_n°
0,Italian,1
1,Italian,1
2,"European, Asian",2
3,"Filipino, American, Italian, Bakery",4
4,American,1


In [40]:
list=df['cuisines'].str.split(expand=True)
list[0].unique()

array(['Italian', 'European,', 'Filipino,', 'American', 'Filipino',
       'Korean,', 'American,', 'Pizza,', 'Taiwanese', 'Japanese,',
       'Japanese', 'Italian,', 'Coffee,', 'Chinese', 'Coffee', 'Seafood,',
       'Korean', 'Singaporean,', 'Vietnamese', 'Latin', 'Healthy',
       'Cafe,', 'Fast', 'Brazilian', 'Argentine,', 'Arabian', 'Cafe',
       'Bakery', 'Tex-Mex,', 'Bar', 'International', 'Pizza', 'French',
       'Arabian,', 'Steak', 'German', 'Brazilian,', 'Sushi,', 'Grill,',
       'Peruvian', 'Peruvian,', 'Seafood', 'North', 'Ice', 'Steak,',
       'Burger,', 'Mexican,', 'Vegetarian,', 'Contemporary', 'Desserts,',
       'Juices,', 'Beverages,', 'Burger', 'Spanish', 'French,', 'Mexican',
       'Thai', 'Indian', 'Mineira,', 'BBQ,', 'Mongolian,', 'Portuguese',
       'Spanish,', 'Contemporary,', 'BBQ', 'Greek', 'Asian,', 'Sushi',
       'Author,', 'Gourmet', 'Beverages', 'Lebanese,', 'Modern',
       'Desserts', 'African,', 'Australian,', 'Middle', 'German,',
       'Malaysi

In [13]:
df=df_raw.copy()

# Business questions

## General information

In [16]:
# 1. Quantos restaurantes únicos estão registrados?

len(df['restaurant_id'].unique())

6929

In [19]:
# 2. Quantos países únicos estão registrados?

len(df['country'].unique())

15

In [22]:
# 3. Quantas cidades únicas estão registradas?

len(df['city'].unique())

125

In [30]:
# 4. Qual o total de avaliações feitas?

print(f'{df["votes"].sum():,.0f}')

4,638,535


In [31]:
# 5. Qual o total de tipos de culinária registrados?

print(f'{df["cuisines_n°"].sum():,.0f}')

18,066


In [ ]:
Pais
1. Qual o nome do país que possui mais cidades registradas?
2. Qual o nome do país que possui mais restaurantes registrados?
3. Qual o nome do país que possui mais restaurantes com o nível de preço igual a 4 registrados?
4. Qual o nome do país que possui a maior quantidade de tipos de culinária distintos?
5. Qual o nome do país que possui a maior quantidade de avaliações feitas?
6. Qual o nome do país que possui a maior quantidade de restaurantes que fazem entrega?
7. Qual o nome do país que possui a maior quantidade de restaurantes que aceitam reservas?
8. Qual o nome do país que possui, na média, a maior quantidade de avaliações registrada?
9. Qual o nome do país que possui, na média, a maior nota média registrada?
10. Qual o nome do país que possui, na média, a menor nota média registrada?
11. Qual a média de preço de um prato para dois por país?

In [ ]:
Cidade
1. Qual o nome da cidade que possui mais restaurantes registrados?
2. Qual o nome da cidade que possui mais restaurantes com nota média acima de 4?
3. Qual o nome da cidade que possui mais restaurantes com nota média abaixo de 2.5?
4. Qual o nome da cidade que possui o maior valor médio de um prato para dois?
5. Qual o nome da cidade que possui a maior quantidade de tipos de culinária distintas?
6. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem reservas?
7. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem entregas?
8. Qual o nome da cidade que possui a maior quantidade de restaurantes que aceitam pedidos online?

In [ ]:
Restaurantes
1. Qual o nome do restaurante que possui a maior quantidade de avaliações?
2. Qual o nome do restaurante com a maior nota média?
3. Qual o nome do restaurante que possui o maior valor de uma prato para duas pessoas?
4. Qual o nome do restaurante de tipo de culinária brasileira que possui a menor média de avaliação?
5. Qual o nome do restaurante de tipo de culinária brasileira, e que é do Brasil, que possui a maior média de avaliação?
6. Os restaurantes que aceitam pedido online são também, na média, os restaurantes que mais possuem avaliações registradas?
7. Os restaurantes que fazem reservas são também, na média, os restaurantes que possuem o maior valor médio de um prato para duas pessoas?
8. Os restaurantes do tipo de culinária japonesa dos Estados Unidos da América possuem um valor médio de prato para duas pessoas maior que
as churrascarias americanas (BBQ)?

In [ ]:
Tipos de Culinária
1. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a maior média de avaliação?
2. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a menor média de avaliação?
3. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a maior média de avaliação?
4. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a menor média de avaliação?
5. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a maior média de avaliação?
6. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a menor média de avaliação?
7. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome dorestaurante com a maior média de avaliação?
8. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome dorestaurante com a menor média de avaliação?
9. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a maior média de avaliação?
10. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a menor média de avaliação?
11. Qual o tipo de culinária que possui o maior valor médio de um prato para duas pessoas?
12. Qual o tipo de culinária que possui a maior nota média?
13. Qual o tipo de culinária que possui mais restaurantes que aceitam pedidos online e fazem entregas?